In [ ]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time
import root_pandas as rpd
from root_pandas import read_root

#mpl.rcParams['figure.figsize'] = 6.2,3.5
#mpl.rcParams['axes.labelsize'] = 17
#mpl.rcParams['lines.linewidth'] = 3
#mpl.rcParams['lines.markersize'] = 6
#mpl.rcParams['legend.fontsize']= 15



mpl.rcParams.update({'font.size': 18})
mpl.rcParams.update({'legend.fontsize': 20})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})

In [ ]:
import numpy as np
import ROOT
from ROOT import TFile
file_pip = TFile.Open('3D_MR_Pt2_piPlus.root')
file_pip.Print()
ydata = {}
xdata = {} 
erry={}
for zbin in range(1,10):
    print(zbin)
    data = file_pip.Get('g_zh_%i_C'%(zbin))
    #data.Print()
    ydata['C_pip_%ibin'%(zbin)] =  np.array(data.GetY(),copy=True)
    xdata['C_pip_%ibin'%(zbin)] =  np.array(data.GetX(),copy=True)
    erry['C_pip_%ibin'%(zbin)] =  np.array(data.GetEY(),copy=True)
    del data
    print(xdata['C_pip_%ibin'%(zbin)])
    data = file_pip.Get('g_zh_%i_Fe'%(zbin))
    #data.Print()
    ydata['Fe_pip_%ibin'%(zbin)] =  np.array(data.GetY(),copy=True)
    xdata['Fe_pip_%ibin'%(zbin)] =  np.array(data.GetX(),copy=True)
    erry['Fe_pip_%ibin'%(zbin)] =  np.array(data.GetEY(),copy=True)
    del data
    data = file_pip.Get('g_zh_%i_Pb'%(zbin))
    #data.Print()
    ydata['Pb_pip_%ibin'%(zbin)] =  np.array(data.GetY(),copy=True)
    xdata['Pb_pip_%ibin'%(zbin)] =  np.array(data.GetX(),copy=True)
    erry['Pb_pip_%ibin'%(zbin)] =  np.array(data.GetEY(),copy=True)
    
    
## pi-    
file_pip.Close()
file_pim = TFile.Open('3D_MR_Pt2_piMinus.root')
file_pim.Print()
for zbin in range(1,10):
    print(zbin)
    data = file_pim.Get('g_zh_%i_C'%(zbin))
    #data.Print()
    ydata['C_pim_%ibin'%(zbin)] =  np.array(data.GetY(),copy=True)
    xdata['C_pim_%ibin'%(zbin)] =  np.array(data.GetX(),copy=True)
    erry['C_pim_%ibin'%(zbin)] =  np.array(data.GetEY(),copy=True)
    
    print(xdata['C_pim_%ibin'%(zbin)])
    del data
    data = file_pim.Get('g_zh_%i_Fe'%(zbin))
    #data.Print()
    ydata['Fe_pim_%ibin'%(zbin)] =  np.array(data.GetY(),copy=True)
    xdata['Fe_pim_%ibin'%(zbin)] =  np.array(data.GetX(),copy=True)
    erry['Fe_pim_%ibin'%(zbin)] =  np.array(data.GetEY(),copy=True)
    del data
    data = file_pim.Get('g_zh_%i_Pb'%(zbin))
    #data.Print()
    ydata['Pb_pim_%ibin'%(zbin)] =  np.array(data.GetY(),copy=True)
    xdata['Pb_pim_%ibin'%(zbin)] =  np.array(data.GetX(),copy=True)
    erry['Pb_pim_%ibin'%(zbin)] =  np.array(data.GetEY(),copy=True)
    
file_pim.Close()



## Rh integrated data for pi+ and pi-

In [ ]:
x_data = np.array([0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95])
y_data = {}
erry_data  ={}

y_data['pip_C'] = np.array([1.16751 , 1.01848 , 0.926135 , 0.876012 , 0.8595 , 0.848906 , 0.832911 , 0.762776 , 0.782656 , 0.698996 ])
erry_data['pip_C'] = np.array([0.00321583 , 0.00161351 , 0.00188183 , 0.00233672 , 0.00297862 , 0.00369811 , 0.00445801 , 0.00520291 , 0.00703654 , 0.00844543])
y_data['pip_Fe'] = np.array([1.18002 , 0.931569 , 0.772485 , 0.695442 , 0.653029 , 0.636352 , 0.609999 , 0.549825 , 0.538038 , 0.440439])
erry_data['pip_Fe'] = np.array([0.00297786 , 0.00136259 , 0.00146705 , 0.00175402 , 0.00214668 , 0.00261719 , 0.00307546 , 0.00352656 , 0.00457231 , 0.00516023])
y_data['pip_Pb'] = np.array([0.980339 , 0.747191 , 0.597805 , 0.522714 , 0.4807 , 0.466481 , 0.446264 , 0.399832 , 0.386629 , 0.277573 ])
erry_data['pip_Pb'] = np.array([0.0028235 , 0.00128281 , 0.00136618 , 0.00163387 , 0.00199655 , 0.0024583 , 0.00293294 , 0.00343855 , 0.00452875 , 0.0049117])

for key in y_data.keys():
    print(len(y_data[key]))
    print(len(x_data))
    
    
x_data_pim = np.array([0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85])

y_data['pim_C'] = np.array([1.16193 , 1.02376 , 0.908824 , 0.884134 , 0.826132 , 0.823838 , 0.802184 , 0.757821 , 0.812838 ])
erry_data['pim_C'] = np.array([ 0.00821278 , 0.00360636 , 0.00431271 , 0.00605682 , 0.00783359 , 0.0109709 , 0.013191 , 0.0149353 , 0.0224106 ]) 
y_data['pim_Fe'] = np.array([1.20089 , 0.943779 , 0.781065 , 0.71245 , 0.654922 , 0.615052 , 0.579656 , 0.541558 , 0.580469   ])
erry_data['pim_Fe'] = np.array([0.00775177 , 0.00298599 , 0.00335861 , 0.00438738 , 0.00557023 , 0.00731394 , 0.00836846 , 0.00938174 , 0.0139264 ]) 
y_data['pim_Pb'] = np.array([1.10823 , 0.82848 , 0.655256 , 0.569205 , 0.514732 , 0.488098 , 0.474604 , 0.37538 , 0.468502  ])
erry_data['pim_Pb'] = np.array([ 0.00803963 , 0.00297901 , 0.00330299 , 0.00429422 , 0.00555248 , 0.00741222 , 0.00917479 , 0.00909736 , 0.0153437 ])

In [ ]:
def applyCut(inputDataframe, cut, text=None):
    nbeforecut = inputDataframe.shape[0]
    if nbeforecut>0:
        cutDataframe = inputDataframe.query(cut)
        if text:
            print (text, cutDataframe.shape[0], ' (%2.2f '%(100.0*cutDataframe.shape[0]/nbeforecut), '%)')
    return cutDataframe
def applyHadronCuts(fullDataframe,name='default',isMC=False): 
    dataframe = fullDataframe
    print ('Entries before cut ', dataframe.shape[0])
    dataframe.eval('inelasticity = Nu/5.014', inplace=True)
    dataframe.eval('E = z*Nu', inplace=True)
    dataframe.eval('P = sqrt(E*E-0.1396*0.1396)', inplace=True)
    dataframe.eval('Pkaon = sqrt(E*E-0.493677*0.493677)', inplace=True)

    dataframe.eval('T = E-0.1396', inplace=True)
    dataframe.eval('pt2 = pt*pt', inplace=True)

    #dataframe = applyCut(dataframe, 'Q2>1.0 and Q2<1.25', ' 1.0 <Q2<1.25 :')
    dataframe = applyCut(dataframe, 'Nu>2.2 and Nu<4.25',  ' 2.2 <Nu<4.25')
    dataframe = applyCut(dataframe, 'P <2.7 ', 'P<2.7 ')
    dataframe = applyCut(dataframe, 'P >0.2 ', 'P>0.2 ')

    dataframe = applyCut(dataframe, 'inelasticity<0.85','inelasticity < 0.85')
    #dataframe = applyCut(dataframe,'(pid==211 & theta_lab_asso>10.0)|(pid_asso==-211 & theta_lab_asso>30 & theta_lab_asso<120)')
    #dataframe = applyCut(dataframe, '(pid==211 & P_asso>0.200) |(pid_asso==-211 & P_asso>0.200) ', 'pi+ P_asso > 200 MeV, pi- P_asso > 300 MeV')
    return dataframe

def applyElectronCuts(fullDataframe,name='default',isMC=False): 
    dataframe = fullDataframe
    print ('Entries before cut ', dataframe.shape[0])
    dataframe.eval('inelasticity = Nu/5.014', inplace=True)
    #dataframe = applyCut(dataframe, 'Q2>1.0 and Q2<1.25', ' 1.0 <Q2<1.25 :')
    dataframe = applyCut(dataframe, 'Nu>2.2 and Nu<4.25',  ' 2.2 <Nu<4.25')
    dataframe = applyCut(dataframe, 'inelasticity<0.85','inelasticity < 0.85')
    return dataframe





In [ ]:
def getRatio(df_A,df_D,df_electron_A,df_electron_D, variable='z',event_cut='Q2>1.0',mybins=None,hadron_cut = ' and P>0',minz=0.05,maxz=1.0,nbins=9):
    
    
    norm_A = df_electron_A.query(event_cut).shape[0]
    norm_D = df_electron_D.query(event_cut).shape[0]
    #print ('Number of electron A', norm_A)
    #print ('Number of electron D', norm_D)
   # 
    #if(mybins==None):  
    #    mybins=np.logspace(np.log10(minz), np.log10(maxz),nbins)
    
    y_A, x_conditional = np.histogram(df_A.query(event_cut+hadron_cut)[variable],bins=mybins)
    y_D, x_conditional = np.histogram(df_D.query(event_cut+hadron_cut)[variable],bins=mybins)
    x_conditional = (x_conditional[1:] + x_conditional[:-1])/2.0
    err_A = np.true_divide(np.sqrt(y_A),y_A)
    err_D = np.true_divide(np.sqrt(y_D),y_D)
    y_A = np.true_divide(y_A,norm_A)
    y_D = np.true_divide(y_D,norm_D)
    ratio_conditional = np.true_divide(y_A,y_D)
    error_conditional = np.multiply(ratio_conditional, np.sqrt(np.power(err_A,2.0) + np.power(err_D,2.0)))
    
    
    return ratio_conditional,error_conditional,x_conditional

## Get data from Ntuples to dataframes

In [ ]:
#this needs to be read in chunks otherwise it uses all memory. (from https://github.com/scikit-hep/root_pandas)
def getdatainChunks(filename,treename):
    dataframe =pd.DataFrame()
    for df in read_root(filename, treename, chunksize=100000) :
        #print df.shape[0]
        dataframe = pd.concat([dataframe,df])
    
    print (dataframe.shape[0])
    return dataframe
    

In [ ]:
df = {}

## GiBUU 

In [ ]:
for target in ['D','C','Fe','Pb']:
    #GiBUU
    print (target)
    print (' hadron')
    df['GiBUU_%s_hadron'%target]= getdatainChunks('GiBUU_SingleHadron_%s.root'%target, '%s_hadron'%target)
    print (' electron')
    df['GiBUU_%s_electron'%target]= getdatainChunks('GiBUU_SingleHadron_%s.root'%target,'%s_electron'%target)
print (df.keys())

In [ ]:
print (df.keys())

## Apply cuts

In [ ]:
for target in ['D','C','Fe','Pb']:
    print (target)
    df['GiBUU_%s_electron'%target] = applyElectronCuts(df['GiBUU_%s_electron'%target], isMC=True)
    df['GiBUU_%s_hadron'%target] = applyHadronCuts(df['GiBUU_%s_hadron'%target], isMC=True)
    print (' ') 


In [ ]:
df['GiBUU_D_hadron'].hist(figsize=(15, 15),density=True,alpha=0.5,bins=100)
plt.show()

In [ ]:
df['GiBUU_Pb_hadron'].hist(figsize=(15, 15),density=True,alpha=0.5,bins=100)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211 and Nu>2.0 and Nu<4.0'), df['GiBUU_D_hadron'].query('pid==211 and Nu>2.0 and Nu<4.0'), 
                        df['GiBUU_%s_electron'%target].query('Nu>2.0 and Nu<4.0'), df['GiBUU_D_electron'].query('Nu>2.0 and Nu<4.0'),
                        mybins=np.linspace(0.1,1.0,20))
    plt.errorbar(x,r,yerr=err,label='This work,  %s'%target,fmt='o',ms=8)
    
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                       mybins=np.linspace(0.1,1.0,20))
    plt.errorbar(x,r,yerr=err,label='This work,  %s'%target,fmt='o',ms=8)
plt.xlabel('$z_{2}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)


plt.tight_layout()
plt.savefig('Rh.png')
#plt.xlim([0.05,0.5])

#plt.xscale('log')
#plt.savefig('ComparisonWithHermes_Log.png')

plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        mybins=np.linspace(0.1,1.0,20))
    plt.errorbar(x,r,yerr=err,label='This work,  %s'%target,fmt='o',ms=8)

plt.xlabel('$z_{2}$',fontsize=18)
plt.ylabel('$R_{2h}$',fontsize=18)


plt.tight_layout()
plt.savefig('Rh.png')
#plt.xlim([0.05,0.5])

#plt.xscale('log')
#plt.savefig('ComparisonWithHermes_Log.png')

plt.show()

## Rh vs z for both pi+ and pi- 

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

ratio = {}

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211 and xf>0'), df['GiBUU_D_hadron'].query('pid==211 and xf>0'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                          mybins=np.linspace(0.05,1.0,10))
    
    plt.errorbar(x,r_pip,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        mybins=np.linspace(0.05,1.0,10))
    
    plt.errorbar(x,r_pim,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    
    
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==2112 and xf>0'), df['GiBUU_D_hadron'].query('pid==2112 and xf>0'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        mybins=np.linspace(0.05,1.0,10))
    
    plt.errorbar(x,r,yerr=err,label=r'p '+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    
    print ('pi+', r_pip)
    print ('pi-', r_pim)
    r = np.true_divide(r_pip,r_pim)
    ratio[target] = r
    
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.0,3.0])

plt.tight_layout()
plt.savefig('Rh.png')

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = '#1f77b4'#'red'
colors['Fe'] = '#2ca02c' #'blue'
colors['Pb']= '#d62728'#'black'
#'#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
ratio = {}

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==321'), df['GiBUU_D_hadron'].query('pid==321'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                       mybins=np.linspace(0.00,1.0,20))
       
    
    plt.errorbar(x,r_pip,yerr=err,label='GiBUU '+'%s'%target,fmt='-',ms=8,color=colors[target],lw=3)


minQ2 = 1.0
maxQ2 = 4.0
minNu = 2.2
maxNu = 4.0
plt.title(r'%2.1f$<Q^{2}<$%2.1f GeV$^{2}$, %2.1f$<\nu<$%2.1f GeV, '%(minQ2,maxQ2,minNu,maxNu),fontsize=20)
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False,fontsize=18)


plt.tight_layout()
plt.savefig('Rh_data_gibuu.png')
plt.savefig('Rh_data_gibuu.pdf')


## Calculate Kaon-to-pion fractions as a function of either momentum or z

In [ ]:
## Number of z bins
zbins = 20

In [ ]:
nkaons, x , _ = plt.hist(df['GiBUU_D_hadron'].query('pid==321')['Pkaon'],bins=zbins,alpha=0.3,range=(0,2.7))
npions, x,  _  = plt.hist(df['GiBUU_D_hadron'].query('pid==211')['P'],bins=zbins,alpha=0.3,range=(0,2.7))
x= (x[1:] + x[:-1])/2.0

kaon_to_pion= np.true_divide(nkaons,npions)
plt.plot(x,kaon_to_pion)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(x,ratio,label='$K^{+}/\pi^{+}$ ratio from Pythia')
plt.fill_between(x,kaon_to_pion*1.5,kaon_to_pion*0.5,alpha=0.3)
#plt.title('$K^{+}/\pi^{+}$ ratio from Pythia',fontsize=20)
plt.xlabel('Momentum [GeV]',fontsize=20)
plt.ylabel('ratio',fontsize=20)
plt.ylim([0,0.35])


peakratio = np.array([0.0111111 , 0.0111111 , 0.0174419 , 0.0444444 , 0.0766667 , 0.128571 , 0.1875 , 0.2125  ])
xpeakratio = np.array([ 0.125, 0.375 , 0.625,0.875,1.125,1.375,1.625,1.875 ])
plt.errorbar(xpeakratio,peakratio,uplims=peakratio-0.05,label='Mass peak ratios from data')
plt.legend(frameon=False)
plt.show()

plt.tight_layout()
plt.savefig('Kplusfraction_momentum.png')
plt.savefig('Kplusfraction_momentum.pdf')


print(ratio)

## Get fraction as a function of z

In [ ]:
nkaons, x , _ = plt.hist(df['GiBUU_D_hadron'].query('pid==321')['z'],bins=zbins,alpha=0.3,range=(0.0,1.0))
npions, x,  _  = plt.hist(df['GiBUU_D_hadron'].query('pid==211')['z'],bins=zbins,alpha=0.3,range=(0.0,1.0))
x= (x[1:] + x[:-1])/2.0

kaon_to_pion = np.true_divide(nkaons,npions)
plt.plot(x,kaon_to_pion)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(x,kaon_to_pion)
plt.title('$K^{+}/\pi^{+}$ ratio from Pythia',fontsize=20)
plt.fill_between(x,kaon_to_pion*2.0,kaon_to_pion*0.5,alpha=0.3)

plt.xlabel('z',fontsize=20)
plt.ylabel('ratio',fontsize=20)
plt.ylim([0,0.35])

plt.tight_layout()
plt.savefig('Kplusfraction_z.png')
plt.savefig('Kplusfraction_z.pdf')


## The signal fraction is given by 
## f = 1/(K/pi + 1)
## so the background fraction is given by: 
## b = 1-f = 1- 1/(K/pi+1)

In [ ]:
rejection_factor = np.array([       1.0, 1.        , 1.      ,   1.    ,     0.99995267 ,0.98168679,
 0.83226166, 0.58285353, 0.37345192 ,0.23785368])

In [ ]:
rejection_factor = np.array([       1.0,1. ,        1.,         1. ,        1. ,        1.,
 1. ,        1.,         0.99999528, 0.99956607 ,0.99345491 ,0.96484801,
 0.89767831 ,0.79514723, 0.67552468, 0.55746647, 0.45239468, 0.36456274,
 0.29378019, 0.23785368])

In [ ]:
## The rejection factor brings the K/pi ratio from
## K/pi to (1-r)K/pi

In [ ]:
plt.figure(figsize=(8,6))
bkg_fraction = 1-1.0/(kaon_to_pion+1)
plt.plot(x,bkg_fraction,label='Before TOF cut')

bkg_fraction = 1-1.0/((1-rejection_factor)*kaon_to_pion+1)
plt.plot(x,bkg_fraction,label='After TOF cut')



#plt.xlabel('z',fontsize=22)
plt.xlabel('Momentum [GeV]',fontsize=22)

plt.ylabel('ratio',fontsize=22)
plt.ylim([0,0.18])
plt.xlim([0.4,2.7])

plt.title('Background fraction',fontsize=20)
plt.legend(loc='best',frameon=False)
plt.tight_layout()

plt.savefig('BackgroundFraction_momentum.png')
plt.savefig('BackgrounFraction_momentum.pdf')


In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = '#1f77b4'#'red'
colors['Fe'] = '#2ca02c' #'blue'
colors['Pb']= '#d62728'#'black'
#'#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',

MR_pion = {}
MR_kaon = {}

binning = np.linspace(0.0,1.0,zbins+1)

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                       mybins=binning)
       
    
    plt.errorbar(x,r_pip,yerr=err,label='$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target],lw=3)
  
    r_kaon, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==321'), df['GiBUU_D_hadron'].query('pid==321'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                       mybins=binning)
       
    
    plt.errorbar(x,r_kaon,yerr=err,label='$K^{+}$ '+'%s'%target,fmt='--',ms=8,color=colors[target],lw=3)

    MR_pion[target] = r_pip
    MR_kaon[target] = r_kaon
    
minQ2 = 1.0
maxQ2 = 4.0
minNu = 2.2
maxNu = 4.0
#plt.title(r'%2.1f$<Q^{2}<$%2.1f GeV$^{2}$, %2.1f$<\nu<$%2.1f GeV, '%(minQ2,maxQ2,minNu,maxNu),fontsize=20)
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False,fontsize=18)
#plt.ylim([0,1.7])
plt.tight_layout()
plt.savefig('Rh_Kaon_and_pion.png')
plt.savefig('Rh_Kaon_and_pion.pdf')



In [ ]:
print(MR_pion['C'])
print(MR_kaon['C'])
print(ratio)
print(len(MR_pion['C']))
print(len(ratio))

In [ ]:
plt.plot(x,rejection_factor)
plt.plot(x, kaon_to_pion)
plt.plot(x,kaon_to_pion*(1-rejection_factor))
plt.fill_between(x,0.5*kaon_to_pion*(1-rejection_factor),2.0*kaon_to_pion*(1-rejection_factor))

## Kaon to pion ratio

In [ ]:
corrected_kaon_to_pion = kaon_to_pion*(1-rejection_factor)
fsignal = 1.0/(corrected_kaon_to_pion+1)
fsignal_up = 1.0/(2*corrected_kaon_to_pion+1)
fsignal_do  =  1.0/(0.5*corrected_kaon_to_pion+1)

In [ ]:
R = {}
R_up = {}
R_do = {}
for target in ['C','Fe','Pb']:
    R[target] = fsignal*MR_pion[target] + (1-fsignal)*MR_kaon[target]
    R_up[target] = fsignal_up*MR_pion[target] + (1-fsignal_up)*MR_kaon[target] 
    R_do[target] = fsignal_do*MR_pion[target] + (1-fsignal_do)*MR_kaon[target]

In [ ]:
fig = plt.figure(figsize=(8,6))


for target in ['C','Fe','Pb']:

    plt.plot(x,MR_pion['Pb'],label='Pure sample')
    plt.plot(x,R[target], label='Contaminated sample')
    plt.fill_between(x,R_do[target],R_up[target], label='Contaminated sample',alpha=0.3)

plt.ylim([0,1.0])
plt.legend(frameon=False)
#plt.xlim([0.3,0.8])

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    plt.plot(x,np.true_divide(MR_pion[target],R[target]),'o-',label=target)
    #plt.fill_between(x,np.true_divide(R_do[target],MR_pion[target]), np.true_divide(R_up[target],MR_pion[target]),alpha=0.3)

plt.ylabel('Signal/(Signal + Background)',fontsize=20)
plt.xlabel('z',fontsize=20)
plt.title('Impact of $K^{+}$ bkg on multiplicity ratio',fontsize=20)
#plt.ylim([1,1.03])
#plt.xlim([0.0,0.96])
plt.legend(loc='best',frameon=False)
print(correction)
plt.tight_layout()
plt.savefig('KaonBKGImpact.png')
plt.savefig('KaonBKGImpact.pdf')



## Comparison to data

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = '#1f77b4'#'red'
colors['Fe'] = '#2ca02c' #'blue'
colors['Pb']= '#d62728'#'black'
#'#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
ratio = {}

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                       mybins=np.linspace(0.00,1.0,10))
    #print(len(x_data))
    #print(len(y_data[target]))
    plt.errorbar(x_data,y_data['pip_%s'%target],yerr=erry_data['pip_%s'%target],color=colors[target],fmt='o',label=target)
    plt.fill_between(x_data, 1.05*y_data['pip_%s'%target],0.95*y_data['pip_%s'%target],color=colors[target],alpha=0.2)
    
    
    plt.errorbar(x,r_pip,yerr=err,label='GiBUU '+'%s'%target,fmt='-',ms=8,color=colors[target],lw=3)
    #plt.errorbar(x_data_pim ,y_data['pim_%s'%target],yerr=erry_data['pim_%s'%target],color=colors[target],fmt='o',label=target)
    #plt.fill_between(x_data_pim, 1.05*y_data['pim_%s'%target],0.95*y_data['pim_%s'%target],color=colors[target],alpha=0.2)
 
    

minQ2 = 1.0
maxQ2 = 4.0
minNu = 2.2
maxNu = 4.0
plt.title(r'%2.1f$<Q^{2}<$%2.1f GeV$^{2}$, %2.1f$<\nu<$%2.1f GeV, '%(minQ2,maxQ2,minNu,maxNu),fontsize=20)
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False,fontsize=18)
plt.ylim([0.0,1.3])

plt.tight_layout()
plt.savefig('Rh_data_gibuu.png')
plt.savefig('Rh_data_gibuu.pdf')



## 3 panel figure

In [ ]:
import csv

FF = {}

errFF_plus = {}
errFF_minus = {}


FF['C_pim'] = []
FF['Fe_pim'] = []
FF['Pb_pim'] = []


errFF_plus['C_pim'] = []
errFF_plus['Fe_pim'] = []
errFF_plus['Pb_pim'] = []

errFF_minus['C_pim'] = []
errFF_minus['Fe_pim'] = []
errFF_minus['Pb_pim'] = []

x_FF = []

with open('clas_for_miguel_68CL_minus.txt') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        row = row[0].split()
        #print(row)
        x_FF.append(float(row[2]))
        FF['C_pim'].append(float(row[3]))
        errFF_minus['C_pim'].append(float(row[4]))
        errFF_plus['C_pim'].append(float(row[5]))

        FF['Fe_pim'].append(float(row[6]))
        errFF_minus['Fe_pim'].append(float(row[7]))
        errFF_plus['Fe_pim'].append(float(row[8]))

        FF['Pb_pim'].append(float(row[9]))
        errFF_minus['Pb_pim'].append(float(row[10]))
        errFF_plus['Pb_pim'].append(float(row[11]))

FF['C_pip'] = []
FF['Fe_pip'] = []
FF['Pb_pip'] = []


errFF_plus['C_pip'] = []
errFF_plus['Fe_pip'] = []
errFF_plus['Pb_pip'] = []

errFF_minus['C_pip'] = []
errFF_minus['Fe_pip'] = []
errFF_minus['Pb_pip'] = []

x_FF = []

with open('clas_for_miguel_68CL_plus.txt') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        row = row[0].split()
        #print(row)
        x_FF.append(float(row[2]))
        FF['C_pip'].append(float(row[3]))
        errFF_minus['C_pip'].append(float(row[4]))
        errFF_plus['C_pip'].append(float(row[5]))

        FF['Fe_pip'].append(float(row[6]))
        errFF_minus['Fe_pip'].append(float(row[7]))
        errFF_plus['Fe_pip'].append(float(row[8]))

        FF['Pb_pip'].append(float(row[9]))
        errFF_minus['Pb_pip'].append(float(row[10]))
        errFF_plus['Pb_pip'].append(float(row[11]))



In [ ]:
fig = plt.figure(figsize=(16,12))


label = {} 
label['C'] = '$C$'
label['Fe'] = '$Fe$'
label['Pb'] = '$Pb$'



colors = {}
colors['C'] = '#1f77b4'
colors['C_pim'] = '#ff7f0e'
colors['Fe']=  '#2ca02c'

colors['Fe_pim'] = '#d62728'
colors['Pb'] = '#9467bd'
colors['Pb_pim']=  '#8c564b'


fig, axs = plt.subplots(1, 3,sharex=True, sharey=True,figsize=(16,7),gridspec_kw={'hspace': 0, 'wspace':0})

for i,target in enumerate(['C','Fe','Pb']):

    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                      mybins=np.linspace(0.0,1.0,10))
    
    lgibuu_pip = axs[i].errorbar(x,y=r_pip,yerr=np.zeros(len(x)),fmt='-',color='black',lw=3,alpha=0.7,label='GiBUU $\pi^{+}$')

    r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        mybins=np.linspace(0.0,1.0,10))
    
 
    lgibuu_pim = axs[i].errorbar(x,y=r_pim,yerr=np.zeros(len(x)),fmt='--',color='black',lw=3,alpha=0.7,label='GiBUU $\pi^{-}$')

        #pi+
    lpip = axs[i].errorbar(x_data-0.01,y_data['pip_%s'%target],yerr=erry_data['pip_%s'%target],color='#1f77b4',fmt='o',
                           ms=6,alpha=0.99,label='$\pi^{+}$')
    #axs[i].fill_between(x_data-0.01, 1.035*y_data['pip_%s'%target],0.965*y_data['pip_%s'%target],color='#1f77b4',alpha=0.2)#,label='$\pi^{+}$')
    axs[i].errorbar(x_data-0.01,y_data['pip_%s'%target],yerr=0.035*y_data['pip_%s'%target],color='#1f77b4',fmt='o',ms=6,alpha=0.7)
    #pi-
    lpim = axs[i].errorbar(x_data_pim+0.01 ,y_data['pim_%s'%target],yerr=erry_data['pim_%s'%target],color='#ff7f0e',fmt='s',
                           lw=3,ms=6,alpha=0.99,label='$\pi^{-}$')
    #axs[i].fill_between(x_data_pim+0.01, 1.055*y_data['pim_%s'%target],0.945*y_data['pim_%s'%target],color='#ff7f0e',alpha=0.2)#,label='$\pi^{-}$')
    axs[i].errorbar(x_data_pim+0.01 ,y_data['pim_%s'%target],yerr=0.055*y_data['pim_%s'%target],color='#ff7f0e',fmt='s',lw=3,ms=6,alpha=0.7)

    axs[i].xaxis.set_label_text('$z$',fontsize=22)
    #axs[i].legend(loc='lower left',ncol=2,frameon=False,fontsize=19)
axs[0].set_title('$C$',fontsize=22)
axs[1].set_title('$Fe$',fontsize=22)
axs[2].set_title('$Pb$',fontsize=22)
axs[0].set_ylim([0.0,1.35])


lben = axs[2].errorbar(x=np.array([0.52,0.62,0.72,0.82,0.92]),y=np.array([0.463033,0.506209,0.458207,0.388899,0.319686]),yerr=np.zeros(len(np.array([0.52,0.62,0.72,0.82,0.92]))),
                       label='Guiot et al.',color='purple',lw=2,alpha=0.5)
axs[1].plot(np.array([0.52,0.62,0.72,0.82,0.92]),np.array([0.613186,0.650574,0.604122,0.535936,0.465189]),
            label='Guiot et al.',color='purple',lw=2,alpha=0.5)
axs[0].plot(np.array([0.52,0.62,0.72,0.82,0.92]),np.array([0.796494,0.820125,0.786835,0.736159,0.682875]), 
            label ='Guiot et al.',color='purple',lw=2,alpha=0.5)

#handles = [lpip,lpim,lgibuu_pim,lgibuu_pim,lben]
# labels=np.array(['GiBUU $\pi^{+}$','GiBUU $\pi^{-}$','Guiot $\pi^{\pm}$','$\pi^{-}$','$\pi^{+}$'])
#handles = [handles[i] for i in [0,1,2,4,3]]
#line_labels =['ben','a','b','c','d']
#fig.legend(handles=handles)





for i,target in enumerate(['C','Fe','Pb']):
    axs[i].errorbar(x_FF[2:10],FF['%s_pip'%target][2:10],label='pi+ %s'%target,color='green')
    axs[i].errorbar(x_FF[2:10],FF['%s_pim'%target][2:10],label ='pi- %s'%target,color='red')
    lzurita_pim = axs[i].fill_between(x_FF[2:10],errFF_minus['%s_pim'%target][2:10],
                                      errFF_plus['%s_pim'%target][2:10],alpha=0.2,label='nFF, $\pi^{-}$',color='red')                 

    lzurita_pip = axs[i].fill_between(x_FF[2:10],errFF_minus['%s_pip'%target][2:10],
                                      errFF_plus['%s_pip'%target][2:10],alpha=0.2,label='nFF, $\pi^{+}$',color='green')

handles = [lpip,lpim, lgibuu_pip, lgibuu_pim,lben,lzurita_pip,lzurita_pim]


fig.legend(handles = handles,     # The line objects,   # The labels for each line
           loc="center right",   # Position of legend
           borderaxespad=0.01,    # Small spacing around legend box
           title="" , # Title for the legend
           frameon=False,
           fontsize=18
          )

#fig.legend(np.array([lpip,lpim]),     # The line objects
#           labels=np.array(['bedn','as']),   # The labels for each line
#           loc="upper right",   # Position of legend
#           borderaxespad=0.05,    # Small spacing around legend box
#           title="" , # Title for the legend
#           frameon=False 
#          )
plt.subplots_adjust(right=0.85)

axs[0].yaxis.set_label_text('$R_{h}$',fontsize=22)


#plt.tight_layout()
plt.savefig('Rh_data_gibuu_3panels.png')
plt.savefig('Rh_data_gibuu_3panels.pdf')


In [ ]:
fig = plt.figure(figsize=(16,12))


label = {} 
label['C'] = '$C$'
label['Fe'] = '$Fe$'
label['Pb'] = '$Pb$'



colors = {}
colors['C'] = '#1f77b4'
colors['C_pim'] = '#ff7f0e'
colors['Fe']=  '#2ca02c'

colors['Fe_pim'] = '#d62728'
colors['Pb'] = '#9467bd'
colors['Pb_pim']=  '#8c564b'


fig, axs = plt.subplots(1, 3,sharex=True, sharey=True,figsize=(16,6),gridspec_kw={'hspace': 0, 'wspace':0})

for i,target in enumerate(['C','Fe','Pb']):

    ratio = np.divide(y_data['pim_%s'%target], y_data['pip_%s'%target][:-1])
    
    err_ratio_pip = np.divide(erry_data['pip_%s'%target][:-1],y_data['pip_%s'%target][:-1])
    err_ratio_pim = np.divide(erry_data['pim_%s'%target],y_data['pim_%s'%target])

    err_ratio = np.sqrt(np.power(err_ratio_pip,2.0)+ np.power(err_ratio_pim,2.0))
    err_ratio = np.multiply(ratio, err_ratio)
    
    
    axs[i].errorbar(x_data_pim, ratio,label='data',yerr=err_ratio)
    
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                      mybins=np.linspace(0.05,1.0,16))
    
    #axs[i].plot(x,r_pip,'-',color='black',label='GiBUU $\pi^{+}$',lw=2)

    r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        mybins=np.linspace(0.05,1.0,16))
    
    ratio_GiBUU = np.divide(r_pim,r_pip)
    axs[i].errorbar(x, ratio_GiBUU,label='GiBUU')
    
    axs[i].xaxis.set_label_text('$z$',fontsize=22)
    axs[i].legend(loc='lower left',ncol=2,frameon=False,fontsize=19)
    axs[i].fill_between(x,0.95,1.05,alpha=0.2)
axs[0].set_title('$C$',fontsize=22)
axs[1].set_title('$Fe$',fontsize=22)
axs[2].set_title('$Pb$',fontsize=22)
axs[0].set_ylim([0.8,1.35])

axs[0].yaxis.set_label_text('$R_{h}$',fontsize=22)

plt.tight_layout()
plt.savefig('Rh_data_gibuu_3panels.png')
plt.savefig('Rh_data_gibuu_3panels.pdf')

In [ ]:
print (ratio.keys())

# momentum

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        mybins=np.linspace(0.2,2.5,10),variable='P')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        mybins=np.linspace(0.2,2.5,10),variable='P')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    

plt.xlabel('Momentum ',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
#plt.ylim([0.0,1.3])

plt.tight_layout()
plt.savefig('Rh_mmentum.png')

## Transverse momentum

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211 and z>0.3and z<0.8'), df['GiBUU_D_hadron'].query('pid==211 and z>0.3 and z<0.8'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        mybins = np.linspace(0.01,1.2,15),variable='pt2')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211 and z>0.3 and z<0.8'), df['GiBUU_D_hadron'].query('pid==-211 and z>0.3 and z<0.8'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        mybins = np.linspace(0.01,1.2,15),variable='pt2')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    

plt.title('0.3<z<0.8',fontsize=22)
plt.xlabel('Transverse momentum squared [GeV$^{2}$]',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
#plt.ylim([0.0,1.3])

#plt.xscale('log')
plt.tight_layout()
plt.savefig('Rh_pt2.png')

## Transverse momentum vs z

In [ ]:
fig, axs = plt.subplots(1, 3,sharex=True, sharey=True,figsize=(16,6),gridspec_kw={'hspace': 0, 'wspace':0})

for i,target in enumerate(['C','Fe','Pb']):
    
    for counter,edges in enumerate([(0.3,0.5), (0.5,0.7),(0.7,1.0)]):
        zquery = ' z>%2.2f and z<%2.2f'%(edges[0],edges[1])
        label = '%2.2f<z<%2.2f'%(edges[0],edges[1])
        #zquery = ' z>0.3 and z<0.8'
        r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211 and' +zquery), df['GiBUU_D_hadron'].query('pid==211 and' + zquery),
                            df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                            mybins = np.linspace(0.01,1.22,10),variable='pt2')
    
        axs[i].set_title(target)
        axs[i].errorbar(x,r,yerr=err,fmt='-',ms=8,label=label)
        axs[i].legend(loc='best',frameon=False)
        axs[i].xaxis.set_label_text('$p_{T}^{2}$ [GeV$^{2}$]',fontsize=22)
        axs[i].yaxis.set_data_interval(0,1.0)
axs[0].yaxis.set_label_text('$R_{h}$',fontsize=22)
plt.tight_layout()
plt.savefig('Multiplicityratio_pt2_3panels.pdf')

## 3x3 panel figure

In [ ]:
print(ydata.keys())
print(xdata['C_pip_1bin'])

## pi+

In [ ]:
fig, axs = plt.subplots(2, 3,sharex=True,sharey='row',figsize=(16,9),gridspec_kw={'hspace': 0, 'wspace':0})


CroninBins_pip = np.logspace(np.log10(0.02),np.log10(1.5),20)#np.array([0.0,0.1,0.25,0.4,0.55,0.75,1.0,1.25])
colors={}
colors['C'] = 'red'#'#1f77b4'
colors['Fe'] = 'blue'#'#ff7f0e'
colors['Pb']=  'black'#'#2ca02c'
for zbin in range(1,10):
    if(zbin==1 or zbin==2 or zbin==3): continue
    col = ((zbin-1)%3)
    row = ((zbin-1)//3)-1
    zquery = ' z>%2.2f and z<%2.2f'%((zbin-1)*0.1,zbin*0.1)
    label = '%2.2f<z<%2.2f'%(((zbin-1)*0.1,zbin*0.1))
    for i,target in enumerate(['C','Fe','Pb']):
    #zquery = ' z>0.3 and z<0.8'
        #print(i)
        r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211 and' +zquery), df['GiBUU_D_hadron'].query('pid==211 and' + zquery),
                            df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                             variable='pt2',mybins=CroninBins_pip)
        #axs[row,col].fill_between(x,r-err,r+err,label='GiBUU %s'%target)
        axs[0,col].set_ylim(bottom=0.0,top=7.1)
        axs[1,col].set_ylim(bottom=0.0,top=4.3)
        axs[row,col].set_xlim(left=0.05,right=1.5)

        ##GIBUU
        axs[row,col].errorbar(x,r,yerr=err,color=colors['%s'%target],lw=2)#,ds='steps')
        axs[row,col].plot(x,r,label='GiBUU %s'%target,color=colors['%s'%target],lw=2)#,ds='steps')

        ##Plot Data
        axs[row,col].errorbar(xdata['%s_pip_%ibin'%(target,zbin)]+0.02*i,ydata['%s_pip_%ibin'%(target,zbin)],yerr=erry['%s_pip_%ibin'%(target,zbin)],ms=10,fmt='o',color=colors['%s'%target],label='%s'%target)
     
        axs[row,col].xaxis.set_label_text('$p_{T}^{2}$ [GeV$^{2}$]',fontsize=22)
        axs[row,0].yaxis.set_label_text('$R_{h}^{\pi^{+}}$',fontsize=24)
        axs[row,col].axhline(y=1.0,color='black',linestyle='--')
        axs[row,col].set_xscale('log')
        #axs[row,col].set_yscale('log')

        zmax = 0.1*zbin
        zmin = 0.1*(zbin-1)
        title=r'%2.2f$<z<$%2.2f'%(zmin,zmax)
        if(row==0):
            axs[row,col].text(0.05, 5.5,title, fontsize=20)
        else:
            axs[row,col].text(0.05, 3.5,title, fontsize=20)
        #axs[row,col].set_title('%2.2f<z<%2.2f'%(zmin,zmax),fontsize=18,loc='right')#,pad=-16.3)

leg = axs[0,0].legend(frameon=False,ncol=2,loc='center left',fontsize=18)  
plt.setp(leg.get_title(),fontsize=20)

plt.tight_layout()
plt.savefig('Cronin_pip.pdf')

## Cronin pi-

In [ ]:
fig, axs = plt.subplots(2, 3,sharex=True,sharey='row',figsize=(16,9),gridspec_kw={'hspace': 0, 'wspace':0})


CroninBins = np.logspace(np.log10(0.02),np.log10(1.5),12)# np.array([0.0,0.1,0.25,0.4,0.55,0.75,1.0,1.25])
colors={}
colors['C'] = 'red'#'#1f77b4'
colors['Fe'] = 'blue'#'#ff7f0e'
colors['Pb']=  'black'#'#2ca02c'
for zbin in range(1,10):
    if(zbin==1 or zbin==2 or zbin==3): continue
    col = ((zbin-1)%3)
    row = ((zbin-1)//3)-1
    zquery = ' z>%2.2f and z<%2.2f'%((zbin-1)*0.1,zbin*0.1)
    label = '%2.2f<z<%2.2f'%(((zbin-1)*0.1,zbin*0.1))
    for i,target in enumerate(['C','Fe','Pb']):
    #zquery = ' z>0.3 and z<0.8'
        #print(i)
        r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211 and' +zquery), df['GiBUU_D_hadron'].query('pid==-211 and' + zquery),
                            df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                             variable='pt2',mybins=CroninBins)
        #axs[row,col].fill_between(x,r-err,r+err,label='GiBUU %s'%target)
        axs[0,col].set_ylim(bottom=0.0,top=6.9)
        axs[1,col].set_ylim(bottom=0.0,top=4.3)
        axs[row,col].set_xlim(left=0.05,right=1.5)

        ##GIBUU
        axs[row,col].errorbar(x,r,yerr=err,color=colors['%s'%target],lw=2)#,ds='steps')
        axs[row,col].plot(x,r,label='GiBUU %s'%target,color=colors['%s'%target],lw=2)#,ds='steps')

        ##Plot Data
        axs[row,col].errorbar(xdata['%s_pim_%ibin'%(target,zbin)]+0.0*i,ydata['%s_pim_%ibin'%(target,zbin)],yerr=erry['%s_pim_%ibin'%(target,zbin)],ms=10,fmt='o',color=colors['%s'%target],label='%s'%target)
     
        axs[row,col].xaxis.set_label_text('$p_{T}^{2}$ [GeV$^{2}$]',fontsize=22)
        axs[row,0].yaxis.set_label_text('$R_{h}^{\pi^{-}}$',fontsize=24)
        axs[row,col].axhline(y=1.0,color='black',linestyle='--')
        axs[row,col].set_xscale('log')
        #axs[row,col].set_yscale('log')

        zmax = 0.1*zbin
        zmin = 0.1*(zbin-1)
        title=r'%2.2f$<z<$%2.2f'%(zmin,zmax)
        if(row==0):
            axs[row,col].text(0.05, 5.5,title, fontsize=20)
        else:
            axs[row,col].text(0.05, 3.5,title, fontsize=20)
        #axs[row,col].set_title('%2.2f<z<%2.2f'%(zmin,zmax),fontsize=18,loc='right')#,pad=-16.3)

leg = axs[0,0].legend(frameon=False,ncol=2,loc='center left',fontsize=18)  
plt.setp(leg.get_title(),fontsize=20)

plt.tight_layout()
plt.savefig('Cronin_pim.pdf')

In [ ]:
fig, axs = plt.subplots(2, 3,sharex=True,sharey='row',figsize=(16,9),gridspec_kw={'hspace': 0, 'wspace':0})


CroninBins_pip = np.array([0.0,0.1,0.25,0.4,0.55,0.75,1.0,1.25])
colors={}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'
for zbin in range(1,10):
    if(zbin==1 or zbin==2 or zbin==3): continue
    col = ((zbin-1)%3)
    row = ((zbin-1)//3)-1
    zquery = ' z>%2.2f and z<%2.2f'%((zbin-1)*0.1,zbin*0.1)
    label = '%2.2f<z<%2.2f'%(((zbin-1)*0.1,zbin*0.1))
    for i,target in enumerate(['C','Fe','Pb']):
    #zquery = ' z>0.3 and z<0.8'
        #print(i)
        r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==111 and' +zquery), df['GiBUU_D_hadron'].query('pid==111 and' + zquery),
                            df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                             variable='pt2',mybins=CroninBins_pip)
        #axs[row,col].fill_between(x,r-err,r+err,label='GiBUU %s'%target)
        axs[0,col].set_ylim(bottom=0.0,top=6.5)
        axs[1,col].set_ylim(bottom=0.0,top=4.2)

        
        ##Plot Data
        axs[row,col].errorbar(x,r,yerr=err,label='GiBUU %s'%target,color=colors['%s'%target],lw=2)#,ds='steps')

        #axs[row,col].legend(frameon=False,ncol=3,loc='lower right',fontsize=15)
        axs[row,col].xaxis.set_label_text('$p_{T}^{2}$ [GeV$^{2}$]',fontsize=22)
        axs[row,0].yaxis.set_label_text('$R_{h}^{\pi^{0}}$',fontsize=22)
        axs[row,col].axhline(y=1.0,color='black',linestyle='--')
        axs[row,col].set_xscale('log')
        #axs[row,col].set_yscale('log')

        zmax = 0.1*zbin
        zmin = 0.1*(zbin-1)
        #axs[row,col].set_title('%2.2f<z<%2.2f'%(zmin,zmax),fontsize=18,loc='right')#,pad=-16.3)
        leg = axs[row,col].legend(frameon=False,ncol=1,loc='upper left',fontsize=16,title='%2.2f<z<%2.2f'%(zmin,zmax))
        plt.setp(leg.get_title(),fontsize=20)
plt.tight_layout()
plt.savefig('Cronin_pi0.pdf')

In [ ]:
fig, axs = plt.subplots(2, 3,sharex=True,sharey='row',figsize=(16,9),gridspec_kw={'hspace': 0, 'wspace':0})


CroninBins_pip = np.array([0.0,0.1,0.25,0.4,0.55,0.75,1.0,1.25])
colors={}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'
for zbin in range(1,10):
    if(zbin==1 or zbin==2 or zbin==3): continue
    col = ((zbin-1)%3)
    row = ((zbin-1)//3)-1
    zquery = ' z>%2.2f and z<%2.2f'%((zbin-1)*0.1,zbin*0.1)
    label = '%2.2f<z<%2.2f'%(((zbin-1)*0.1,zbin*0.1))
    for i,target in enumerate(['C','Fe','Pb']):
    #zquery = ' z>0.3 and z<0.8'
        #print(i)
        r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==2212 and' +zquery), df['GiBUU_D_hadron'].query('pid==2212 and' + zquery),
                            df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                             variable='pt2',mybins=CroninBins_pip)
        #axs[row,col].fill_between(x,r-err,r+err,label='GiBUU %s'%target)
        axs[0,col].set_ylim(bottom=0.0,top=6.5)
        axs[1,col].set_ylim(bottom=0.0,top=4.2)

        
        ##Plot Data
        axs[row,col].errorbar(x,r,yerr=err,label='GiBUU %s'%target,color=colors['%s'%target],lw=2)#,ds='steps')

        #axs[row,col].legend(frameon=False,ncol=3,loc='lower right',fontsize=15)
        axs[row,col].xaxis.set_label_text('$p_{T}^{2}$ [GeV$^{2}$]',fontsize=22)
        axs[row,0].yaxis.set_label_text('$R_{h}^{p}$',fontsize=22)
        axs[row,col].axhline(y=1.0,color='black',linestyle='--')
        axs[row,col].set_xscale('log')
        #axs[row,col].set_yscale('log')

        zmax = 0.1*zbin
        zmin = 0.1*(zbin-1)
        #axs[row,col].set_title('%2.2f<z<%2.2f'%(zmin,zmax),fontsize=18,loc='right')#,pad=-16.3)
        leg = axs[row,col].legend(frameon=False,ncol=1,loc='upper left',fontsize=16,title='%2.2f<z<%2.2f'%(zmin,zmax))
        plt.setp(leg.get_title(),fontsize=20)
plt.tight_layout()
plt.savefig('Cronin_p.pdf')

## Ratio

In [ ]:
fig, axs = plt.subplots(2, 3,sharex=True,sharey='row',figsize=(16,9),gridspec_kw={'hspace': 0, 'wspace':0})


#CroninBins_pip = np.array([0.0,0.1,0.25,0.4,0.55,0.75,1.0,1.25])
colors={}
colors['C'] = '#1f77b4'
colors['Fe'] = '#ff7f0e'
colors['Pb']=  '#2ca02c'
for zbin in range(1,10):
    if(zbin==1 or zbin==2 or zbin==3): continue
    col = ((zbin-1)%3)
    row = ((zbin-1)//3)-1
    zquery = ' z>%2.2f and z<%2.2f'%((zbin-1)*0.1,zbin*0.1)
    label = '%2.2f<z<%2.2f'%(((zbin-1)*0.1,zbin*0.1))
    for i,target in enumerate(['C','Fe','Pb']):
        ##Plot Data
        
        ratio = np.divide(ydata['%s_pim_%ibin'%(target,zbin)],ydata['%s_pip_%ibin'%(target,zbin)])
        
        
        err_ratio_pip = np.divide(erry['%s_pip_%ibin'%(target,zbin)],ydata['%s_pip_%ibin'%(target,zbin)])
        err_ratio_pim = np.divide(erry['%s_pim_%ibin'%(target,zbin)],ydata['%s_pim_%ibin'%(target,zbin)])

        err_ratio = np.sqrt(np.power(err_ratio_pip,2.0)+ np.power(err_ratio_pim,2.0))
        err_ratio = np.multiply(ratio, err_ratio)
        axs[row,col].set_xlim(left=0.05,right=1.2)

        axs[row,col].fill_between(xdata['%s_pim_%ibin'%(target,zbin)]+0.0, 0.9,1.1,alpha=0.2,color='black')
        axs[row,col].errorbar(xdata['%s_pim_%ibin'%(target,zbin)]+0.0*i,ratio,yerr=err_ratio,ms=10,fmt='-o',color=colors['%s'%target],label='%s'%target)
        
        axs[row,col].xaxis.set_label_text('$p_{T}^{2}$ [GeV$^{2}$]',fontsize=22)
        axs[row,col].set_xscale('log')

        #axs[row,col].fill_between(xdata['%s_pip_%ibin'%(target,zbin)][:-1]+0.02*i,0.96*ydata['%s_pip_%ibin'%(target,zbin)][:-1], 1.04*ydata['%s_pip_%ibin'%(target,zbin)][:-1], color=colors['%s'%target])


In [ ]:
fig, axs = plt.subplots(2, 3,sharex=True,sharey='row',figsize=(16,9),gridspec_kw={'hspace': 0, 'wspace':0})


CroninBins_pip = np.array([0.0,0.1,0.25,0.4,0.55,0.75,1.0,1.25])
colors={}
colors['C'] = 'red'#'#1f77b4'
colors['Fe'] = 'blue'#'#ff7f0e'
colors['Pb']=  'black'#'#2ca02c'
for zbin in range(1,10):
    if(zbin==1 or zbin==2 or zbin==3): continue
    col = ((zbin-1)%3)
    row = ((zbin-1)//3)-1
    zquery = ' z>%2.2f and z<%2.2f'%((zbin-1)*0.1,zbin*0.1)
    label = '%2.2f<z<%2.2f'%(((zbin-1)*0.1,zbin*0.1))
    for i,target in enumerate(['C','Fe','Pb']):
    #zquery = ' z>0.3 and z<0.8'
        #print(i)
        rpim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211 and' +zquery), df['GiBUU_D_hadron'].query('pid==-211 and' + zquery),
                            df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                             variable='pt2',mybins=CroninBins_pip)
        rpip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211 and' +zquery), df['GiBUU_D_hadron'].query('pid==211 and' + zquery),
                            df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                             variable='pt2',mybins=CroninBins_pip)
        ratio = np.divide(rpip,rpim)
        axs[row,col].errorbar(x,ratio,color=colors['%s'%target],label='%s'%target)
        axs[row,col].fill_between(x, 0.9,1.1,alpha=0.6,color='black')


plt.legend()
plt.show()

## Broadening

In [ ]:
broadening_pip = []
mass_number = [12,52,208]
mass_number_cubicroot = [2.28, 3.73,5.92]
fig = plt.figure(figsize=(8,6))
broadening_pim= []

selection = 'z>0.5 and z<0.6 and Nu<3 and Nu>2 and Q2<2 and Q2>1'

ref = df['GiBUU_D_hadron'].query('pid==211 and '+selection)['pt2'].mean()
for target in ['C','Fe','Pb']:
    A = df['GiBUU_%s_hadron'%target].query('pid==211 and '+selection)['pt2'].mean()
    print (A-ref)
    #print A
    broadening_pip.append(A-ref)

plt.plot(mass_number_cubicroot,broadening_pip,'o',ms=10,alpha=0.8,label='$\pi^{+}$')


selection = 'z>0.5 and z<0.6 and Nu<4 and Nu>3 and Q2<2 and Q2>1'
broadening_pip= []
ref = df['GiBUU_D_hadron'].query('pid==211 and '+selection)['pt2'].mean()
for target in ['C','Fe','Pb']:
    A = df['GiBUU_%s_hadron'%target].query('pid==211 and '+selection)['pt2'].mean()
    print (A-ref)
    #print A
    broadening_pip.append(A-ref)

plt.plot(mass_number_cubicroot,broadening_pip,'o',ms=10,alpha=0.8,label='3-4')


    
#print broadening
#ref = df['GiBUU_D_hadron'].query('pid==-211 and '+selection)['pt2'].mean()
#for target in ['C','Fe','Pb']:
#    A = df['GiBUU_%s_hadron'%target].query('pid==-211 and '+selection)['pt2'].mean()
#    print A-ref
#    broadening_pim.append(A-ref)
    
#plt.plot(mass_number_cubicroot,broadening_pim,'o',ms=10,alpha=0.8,label='$\pi^{-}$')


plt.title(r'$0.5<z<0.6$ , $2.0<\nu<3.0$',fontsize=22)
plt.ylabel('$p_{T}^{2}$ broadening [GeV$^{2}$]',fontsize=22)
plt.xlabel('Mass number $^{1/3}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.0,0.06])

#plt.xscale('log')
plt.tight_layout()
plt.savefig('Broadening.png')

## Kinetic energy

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

for target in ['C','Fe','Pb']:
    r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211 and P>0.2'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
                        minz=0.1,maxz=2.5,nbins=20,variable='T')
    
    plt.errorbar(x,r,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    #r, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211 and P>0.2'), 
    #                    df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'],
    #                    minz=0.1,maxz=2.5,nbins=10,variable='T')
    
    #plt.errorbar(x,r,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    

plt.xlabel('Kinetic Energy',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
#plt.ylim([0.0,1.3])

plt.xscale('log')
plt.tight_layout()
plt.savefig('Rh_mmentum.png')

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.85
maxQ2 = 2.40
minNu = 2.2
maxNu = 3.0

ratio = {}

event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
print (event_selection)

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
                        minz=0.05,maxz=1.0,nbins=10)
    
    plt.errorbar(x,r_pip,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
                        minz=0.05,maxz=1.0,nbins=10)
    
    plt.errorbar(x,r_pim,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    
    print ('pi+', r_pip)
    print ('pi-', r_pim)
    print (np.true_divide(r_pip,r_pim))
    r = np.true_divide(r_pip,r_pim)
    ratio[target] = r

plt.title(r'%2.2f$<Q^{2}<$%2.2f GeV$^{2}$, %2.2f$<\nu<$%2.2f GeV'%(minQ2,maxQ2,minNu,maxNu),fontsize=22)
    
plt.xlabel('$z$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.0,1.3])

plt.tight_layout()
plt.savefig('Rh.png')

In [ ]:
fig = plt.figure(figsize=(8,6))

for target in ['C','Fe','Pb']:
    plt.plot(x,ratio[target],color=colors[target],label=target)
    plt.xlabel('$z$',fontsize=22)
    plt.ylabel('Ratio of $\pi^{+}$ to $\pi^{-}$',fontsize=22) 
    plt.legend(loc='best',ncol=3,frameon=False)
    #plt.ylim([0.0,1.3])
    
    plt.tight_layout()
    plt.savefig('Rh_pimtopip.png')

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.0
maxQ2 = 1.2


#event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
event_selection = 'Q2>%2.2f and Q2<%2.2f'%(minQ2,maxQ2)

print (event_selection)

for target in ['C','Fe','Pb']:
    r_pip, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==211'), df['GiBUU_D_hadron'].query('pid==211'), 
                        df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
                        minz=0.2,maxz=3.0,nbins=20,variable='P')
    
    plt.errorbar(x,r_pip,yerr=err,label=r'$\pi^{+}$'+'%s'%target,fmt='-',ms=8,color=colors[target])
    
    
    #r_pim, err,x = getRatio(df['GiBUU_%s_hadron'%target].query('pid==-211'), df['GiBUU_D_hadron'].query('pid==-211'), 
    #                    df['GiBUU_%s_electron'%target], df['GiBUU_D_electron'], event_cut = event_selection,
    #                    minz=0.2,maxz=3.0,nbins=20,variable='P')
    
    #plt.errorbar(x,r_pim,yerr=err,label=r'$\pi^{-}$'+'%s'%target,fmt='-',ms=8,ls='--',color=colors[target])
    
    ##print 'pi+', r_pip
    ##print 'pi-', r_pim
    #print np.true_divide(r_pip,r_pim)

plt.title(r'%2.2f$<Q^{2}<$%2.2f GeV$^{2}$'%(minQ2,maxQ2))
    
plt.xlabel('$P [GeV]$',fontsize=22)
plt.ylabel('$R_{h}$',fontsize=22)
plt.legend(loc='best',ncol=3,frameon=False)
plt.ylim([0.1,2.5])
#plt.xscale('log')
plt.tight_layout()
plt.savefig('Rh.png')

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.85
maxQ2 = 2.40
minNu = 2.2
maxNu = 3.0

#event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
event_selection = 'Q2>%2.2f and Q2<%2.2f'%(minQ2,maxQ2)

print (event_selection)



plt.vlines(0.2, 0,3.0,color='red')

#plt.xscale('log')
plt.hist(df['GiBUU_D_hadron'].query('pid==211 and P>0.2')['T'],bins=50,alpha=0.3,density=2.0)
for target in ['D','C','Fe','Pb']:
    plt.hist(df['GiBUU_%s_hadron'%target].query('pid==211 and P>0.2')['T'],histtype='step',bins=50,alpha=0.8,density=2.0)

In [ ]:
fig = plt.figure(figsize=(8,6))


colors = {}
colors['C'] = 'red'
colors['Fe'] = 'blue'
colors['Pb']= 'black'

minQ2 = 1.85
maxQ2 = 2.40
minNu = 2.2
maxNu = 3.0

#event_selection = 'Q2>%2.2f and Q2<%2.2f and Nu>%2.2f and Nu<%2.2f'%(minQ2,maxQ2,minNu,maxNu)
event_selection = 'Q2>%2.2f and Q2<%2.2f'%(minQ2,maxQ2)

print (event_selection)

plt.hist(df['GiBUU_D_hadron'].query('pid==211')['theta_lab'],bins=20,alpha=0.3,density=2.0)
for target in ['D','C','Fe','Pb']:
    plt.hist(df['GiBUU_%s_hadron'%target].query('pid==211')['theta_lab'],histtype='step',bins=20,alpha=0.3,density=2.0)

In [ ]:
print(df.keys())

In [ ]:
df['GiBUU_hadron']